In [5]:
import tensorflow as tf
import larq as lq

In [6]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between -1 and 1
train_images, test_images = train_images / 127.5 - 1, test_images / 127.5 - 1

In [7]:
# All quantized layers except the first will use the same options
kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip")

model = tf.keras.models.Sequential()

# In the first layer we only quantize the weights and not the input
model.add(lq.layers.QuantConv2D(32, (3, 3),
                                use_bias=False,
                                input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Flatten())

model.add(lq.layers.QuantDense(64, use_bias=False))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(lq.layers.QuantDense(10, use_bias=False))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Activation("softmax"))

In [8]:
lq.models.summary(model)

+sequential stats---------------------------------------------------------------------+
| Layer                  Input prec.           Outputs  # 32-bit  Memory  32-bit MACs |
|                              (bit)                         x 1    (kB)              |
+-------------------------------------------------------------------------------------+
| quant_conv2d                     -  (-1, 26, 26, 32)       288    1.12       194688 |
| max_pooling2d                    -  (-1, 13, 13, 32)         0       0            0 |
| batch_normalization              -  (-1, 13, 13, 32)        96    0.38            0 |
| quant_conv2d_1                   -  (-1, 11, 11, 64)     18432   72.00      2230272 |
| max_pooling2d_1                  -    (-1, 5, 5, 64)         0       0            0 |
| batch_normalization_1            -    (-1, 5, 5, 64)       192    0.75            0 |
| quant_conv2d_2                   -    (-1, 3, 3, 64)     36864  144.00       331776 |
| batch_normalization_2         

In [9]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, batch_size=64, epochs=6)

test_loss, test_acc = model.evaluate(test_images, test_labels)

Train on 60000 samples
Epoch 1/6
60000/60000 [==============================] - 41s 682us/sample - loss: 0.5650 - accuracy: 0.9538
Epoch 2/6
60000/60000 [==============================] - 38s 628us/sample - loss: 0.4884 - accuracy: 0.9743
Epoch 3/6
60000/60000 [==============================] - 38s 634us/sample - loss: 0.4714 - accuracy: 0.9786
Epoch 4/6
60000/60000 [==============================] - 38s 635us/sample - loss: 0.4639 - accuracy: 0.9807
Epoch 5/6
60000/60000 [==============================] - 38s 638us/sample - loss: 0.4560 - accuracy: 0.9824
Epoch 6/6
60000/60000 [==============================] - 38s 639us/sample - loss: 0.4532 - accuracy: 0.9821
10000/1 [================================================================================================================================================================================================================================================================================================================================

In [10]:
print(f"Test accuracy {test_acc * 100:.2f} %")

Test accuracy 98.72 %
